# IBM Applied Data Science Capstone Course by Coursera
### Week 5 Final Report
**_Opening a New Shopping Mall in London**
- Build a dataframe of neighborhoods in London by web scraping the data from Wikipedia page
- Get the geographical coordinates of the neighborhoods
- Obtain the venue data for the neighborhoods from Foursquare API
- Explore and cluster the neighborhoods
- Select the best cluster to open a new shopping mall
***
### 1. Import libraries

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

!pip install geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

print("Libraries imported.")

     |████████████████████████████████| 92kB 6.7MB/s eta 0:00:011
Libraries imported.


### 2. Scrap data from Wikipedia page into a DataFrame

In [32]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Districts_of_Paris").text

In [33]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [34]:
# create a list to store neighborhood data
neighborhoodList = []

In [35]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [38]:
# create a new DataFrame from the list
paris_df = pd.DataFrame({"Neighborhood": neighborhoodList})

paris_df.head(10)

,Neighborhood
0,Historical quarters of Paris
1,Revolutionary sections of Paris
2,Batignolles
3,"Belleville, Paris"
4,Bercy
5,Cité des Fleurs
6,Cour des miracles
7,Épinettes
8,Faubourg Saint-Antoine
9,Faubourg Saint-Germain


In [39]:
# print the number of rows of the dataframe
paris_df.shape

(31, 1)

### 3. Get the geographical coordinates

In [40]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Paris'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [41]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in paris_df["Neighborhood"].tolist() ]

In [13]:
coords

[[3.0576900000000364, 101.74388000000005],
 [3.158092785454847, 101.74870260727259],
 [3.1903500000000236, 101.62545000000006],
 [3.1039100000000417, 101.71226000000007],
 [3.072620000000029, 101.71471000000008],
 [3.08280000000002, 101.72281000000004],
 [3.1292000000000257, 101.67844000000008],
 [3.1347800000000348, 101.67262000000005],
 [3.111020000000053, 101.66283000000004],
 [3.09898000000004, 101.73499000000004],
 [3.09898000000004, 101.73499000000004],
 [3.12916000000007, 101.68406000000004],
 [3.147770000000037, 101.70855000000006],
 [3.057800000000043, 101.68965000000009],
 [3.1434800000000678, 101.64433000000008],
 [3.1520079194122164, 101.70102829894834],
 [3.129290000000026, 101.69892000000004],
 [3.17381000000006, 101.68276000000009],
 [3.061870000000056, 101.74675000000008],
 [3.163590000000056, 101.69811000000004],
 [3.147980000000075, 101.66798000000006],
 [3.1413156714557724, 101.62607997785557],
 [3.1413156714557724, 101.62607997785557],
 [3.155238081462032, 101.70152

In [42]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [43]:
# merge the coordinates into the original dataframe
paris_df['Latitude'] = df_coords['Latitude']
paris_df['Longitude'] = df_coords['Longitude']

In [44]:
# check the neighborhoods and the coordinates
print(paris_df.shape)
paris_df

(31, 3)


,Neighborhood,Latitude,Longitude
0,Historical quarters of Paris,42.646260,-90.598530
1,Revolutionary sections of Paris,48.857170,2.341400
2,Batignolles,48.883330,2.316670
3,"Belleville, Paris",48.870180,2.384230
4,Bercy,48.834880,2.384590
5,Cité des Fleurs,48.892612,2.320326
6,Cour des miracles,45.748650,4.881890
7,Épinettes,48.842962,2.325298
8,Faubourg Saint-Antoine,48.850940,2.375670
9,Faubourg Saint-Germain,48.857788,2.323823


In [45]:
# save the DataFrame as CSV file
paris_df.to_csv("paris_df.csv", index=False)

### 4. Create a map of Paris with neighborhoods superimposed on top

In [46]:
# get the coordinates of Kuala Lumpur
address = 'Paris'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Paris {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Paris 48.8566101, 2.3514992.


In [47]:
# create map of Paris using latitude and longitude values
map_paris = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(paris_df['Latitude'], paris_df['Longitude'], paris_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_paris)  
    
map_paris

In [48]:
# save the map as HTML file
map_paris.save('map_paris.html')

### 5. Use the Foursquare API to explore the neighborhoods

In [49]:
# define Foursquare Credentials and Version
CLIENT_ID = 'YY3XMMNYSXY0RKFIOKBGPSPZFFRO3OSQQN5S2S0ZAJ1AFU1Y' # your Foursquare ID
CLIENT_SECRET = 'BJLHJFHHF1MP4JAPYA55GRZBW1LTVDPCEQJGMSZIB0Y3VR0D' # your Foursquare Secret
VERSION = '20200103' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YY3XMMNYSXY0RKFIOKBGPSPZFFRO3OSQQN5S2S0ZAJ1AFU1Y
CLIENT_SECRET:BJLHJFHHF1MP4JAPYA55GRZBW1LTVDPCEQJGMSZIB0Y3VR0D


**Now, let's get the top 100 venues that are within a radius of 2000 meters.**

In [50]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(paris_df['Latitude'], paris_df['Longitude'], paris_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [52]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head(2)

(2747, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Historical quarters of Paris,42.64626,-90.59853,Kuepers III,42.651051,-90.580334,Bar
1,Revolutionary sections of Paris,48.85717,2.34140,Chez Nous,48.855823,2.340271,Wine Bar


**Let's check how many venues were returned for each neighorhood**

In [53]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Batignolles,100,100,100,100,100,100
"Belleville, Paris",100,100,100,100,100,100
Bercy,100,100,100,100,100,100
Cité des Fleurs,100,100,100,100,100,100
Cour des miracles,44,44,44,44,44,44
Faubourg Saint-Antoine,100,100,100,100,100,100
Faubourg Saint-Germain,100,100,100,100,100,100
Front de Seine,100,100,100,100,100,100
Goutte d'Or,100,100,100,100,100,100


**Let's find out how many unique categories can be curated from all the returned venues**

In [54]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 195 uniques categories.


In [55]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Bar', 'Wine Bar', 'Restaurant', 'Plaza', 'Hotel', 'Bridge',
       'Cocktail Bar', 'Pedestrian Plaza', 'Theater', 'Exhibit',
       'French Restaurant', 'Cosmetics Shop', 'Lebanese Restaurant',
       'Bookstore', 'Art Museum', 'Burger Joint', 'Sandwich Place',
       'Modern European Restaurant', 'Ice Cream Shop',
       'Seafood Restaurant', 'Creperie', 'Beer Bar', 'Liquor Store',
       'Toy / Game Store', 'Electronics Store', 'Church', 'Tea Room',
       'Historic Site', 'Bistro', 'Park', 'Comic Shop',
       'Indie Movie Theater', 'Clothing Store', 'Japanese Restaurant',
       'Garden', 'Tapas Restaurant', 'Souvenir Shop', 'Pastry Shop',
       'Chinese Restaurant', 'Italian Restaurant',
       'Furniture / Home Store', 'Bakery', 'Art Gallery',
       'Udon Restaurant', 'Gourmet Shop', 'Wine Shop',
       'Szechuan Restaurant', 'Fountain', 'Dessert Shop',
       'Gym / Fitness Center'], dtype=object)

In [56]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

False

### 6. Analyze Each Neighborhood

In [57]:
# one hot encoding
paris_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
paris_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [paris_onehot.columns[-1]] + list(paris_onehot.columns[:-1])
paris_onehot = paris_onehot[fixed_columns]

print(paris_onehot.shape)
paris_onehot.head()

(2747, 196)


,Neighborhoods,African Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auvergne Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Basque Restaurant,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Bridge,Bubble Tea Shop,Burger Joint,Bus Stop,Café,Cambodian Restaurant,Canal,Candy Store,Cemetery,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Corsican Restaurant,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Doner Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Service,Food Truck,Fountain,French Restaurant,Furniture / Home Store,Garden,Gastropub,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Island,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jiangxi Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Library,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Multiplex,Museum,Music Store,Music Venue,Nightclub,Noodle House,Opera House,Outdoor Sculpture,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pharmacy,Pizza Place,Planetarium,Playground,Plaza,Pool,Pop-Up Shop,Portuguese Restaurant,Provençal Restaurant,Pub,Radio Station,Record Shop,Recording Studio,Restaurant,Roof Deck,Russian Restaurant,Salad Place,Sandwich Place,Savoyard Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shopping Mall,Shopping Plaza,Soccer Field,Southwestern French Restaurant,Souvenir Shop,Spa,Speakeasy,Sports Bar,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Park Ride / Attraction,Toy / Game Store,Track,Trail,Tram Station,Trattoria/Osteria,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Vineyard,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Historical quarters of Paris,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Revolutionary sections of Paris,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,Revolutionary sections of Paris,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,

**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [58]:
paris_grouped = paris_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(paris_grouped.shape)
paris_grouped

(31, 196)


,Neighborhoods,African Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auvergne Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Basque Restaurant,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Bridge,Bubble Tea Shop,Burger Joint,Bus Stop,Café,Cambodian Restaurant,Canal,Candy Store,Cemetery,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Corsican Restaurant,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Doner Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Service,Food Truck,Fountain,French Restaurant,Furniture / Home Store,Garden,Gastropub,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Island,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jiangxi Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Library,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Multiplex,Museum,Music Store,Music Venue,Nightclub,Noodle House,Opera House,Outdoor Sculpture,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pharmacy,Pizza Place,Planetarium,Playground,Plaza,Pool,Pop-Up Shop,Portuguese Restaurant,Provençal Restaurant,Pub,Radio Station,Record Shop,Recording Studio,Restaurant,Roof Deck,Russian Restaurant,Salad Place,Sandwich Place,Savoyard Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shopping Mall,Shopping Plaza,Soccer Field,Southwestern French Restaurant,Souvenir Shop,Spa,Speakeasy,Sports Bar,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Park Ride / Attraction,Toy / Game Store,Track,Trail,Tram Station,Trattoria/Osteria,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Vineyard,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Batignolles,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.000000,0.00,0.00,0.00,0.050000,0.040000,0.01,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.02,0.02,0.000000,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.02,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.200000,0.00,0.000000,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.00,0.01,0.00,0.00,0.100000,0.01,0.00,0.00,0.01,0.00,0.00,0.070000,0.030000,0.000000,0.00,0.00,0.01,0.00,0.01,0.00,0.01,0.02,0.00,0.00,0.000000,0.01,0.00,0.01,0.00,0.000000,0.00,0.00,0.00,0.00,0.010000,0.00,0.00,0.00,0.01,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.010000,0.00,0.00,0.010000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.010000,0.01,0.01,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.000000,0.01,0.00,0.01,0.00,0.01,0.00,0.02,0.00,0.02,0.02,0.00,0.00,0.00,0.00,0.000000,0.00,0.01,0.00,0.00,0.00,0.00,0.030000,0.00,0.00,0.00
1,"Belleville, Paris",0.00,0.00,0.00,0.00,0.01,0.01,0.02,0.000000,0.00,0.01,0.00,0.0

In [59]:
len(paris_grouped[paris_grouped["Shopping Mall"] > 0])

3

**Create a new DataFrame for Shopping Mall data only**

In [60]:
paris_mall = paris_grouped[["Neighborhoods","Shopping Mall"]]

In [61]:
paris_mall.head()

,Neighborhoods,Shopping Mall
0,Batignolles,0.0
1,"Belleville, Paris",0.0
2,Bercy,0.0
3,Cité des Fleurs,0.0
4,Cour des miracles,0.0


### 7. Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Paris into 3 clusters.

In [62]:
# set number of clusters
kclusters = 3

paris_clustering = paris_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(paris_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return_n_iter=True)


array([2, 2, 2, 2, 2, 2, 2, 0, 2, 0], dtype=int32)

In [63]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
paris_merged = paris_mall.copy()

# add clustering labels
paris_merged["Cluster Labels"] = kmeans.labels_

In [64]:
paris_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
paris_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Batignolles,0.0,2
1,"Belleville, Paris",0.0,2
2,Bercy,0.0,2
3,Cité des Fleurs,0.0,2
4,Cour des miracles,0.0,2


In [65]:
# merge paris_grouped with toronto_data to add latitude/longitude for each neighborhood
paris_merged = paris_merged.join(paris_df.set_index("Neighborhood"), on="Neighborhood")

print(paris_merged.shape)
paris_merged.head() # check the last columns!

(31, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Batignolles,0.0,2,48.883330,2.316670
1,"Belleville, Paris",0.0,2,48.870180,2.384230
2,Bercy,0.0,2,48.834880,2.384590
3,Cité des Fleurs,0.0,2,48.892612,2.320326
4,Cour des miracles,0.0,2,45.748650,4.881890


In [66]:
# sort the results by Cluster Labels
print(paris_merged.shape)
paris_merged.sort_values(["Cluster Labels"], inplace=True)
paris_merged

(31, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
7,Front de Seine,0.01,0,48.849310,2.285780
9,Grenelle,0.01,0,48.848420,2.291640
12,"Javel, France",0.01,0,48.843870,2.286130
0,Batignolles,0.00,2,48.883330,2.316670
28,The Marais,0.00,2,48.858760,2.360870
27,Saint-Germain-des-Prés,0.00,2,48.853770,2.333310
26,Revolutionary sections of Paris,0.00,2,48.857170,2.341400
25,Quartier des Grandes-Carrières,0.00,2,48.890880,2.331020
24,Quartier de La Chapelle,0.00,2,48.884150,2.357095
23,Quartier Pigalle,0.00,2,48.882031,2.337576


**Finally, let's visualize the resulting clusters**

In [67]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(paris_merged['Latitude'], paris_merged['Longitude'], paris_merged['Neighborhood'], paris_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [68]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 8. Examine Clusters

#### Cluster 0

In [69]:
paris_merged.loc[paris_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
7,Front de Seine,0.01,0,48.84931,2.28578
9,Grenelle,0.01,0,48.84842,2.29164
12,"Javel, France",0.01,0,48.84387,2.28613


#### Cluster 1

In [70]:
paris_merged.loc[paris_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude


#### Cluster 2

In [71]:
paris_merged.loc[paris_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Batignolles,0.0,2,48.883330,2.316670
28,The Marais,0.0,2,48.858760,2.360870
27,Saint-Germain-des-Prés,0.0,2,48.853770,2.333310
26,Revolutionary sections of Paris,0.0,2,48.857170,2.341400
25,Quartier des Grandes-Carrières,0.0,2,48.890880,2.331020
24,Quartier de La Chapelle,0.0,2,48.884150,2.357095
23,Quartier Pigalle,0.0,2,48.882031,2.337576
22,Quartier Asiatique,0.0,2,48.852030,2.334490
21,Quarters of Paris,0.0,2,42.646260,-90.598530
20,Petit-Montrouge,0.0,2,48.826420,2.325200


#### Observations:
All of the shopping malls are concentrated in the central area of Paris, with the highest number in cluster 2. and moderate number in cluster 0. On the other hand, cluster 1 has no shopping malls in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 2 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb area still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 1 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 0 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 2 which already have high concentration of shopping malls and suffering from intense competition.